In [121]:
%reset -f
%load_ext autoreload
%autoreload 2


import numpy as np
from pint import UnitRegistry
ureg = UnitRegistry()
Q = ureg.Quantity

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Handling Common Values belonging to Specific E-Series

In [122]:
# Generate list of all possible significands and mappings to specific component value series
# https://eepower.com/resistor-guide/resistor-standards-and-codes/resistor-values/

def gen_significand3_map():
    """
    Different value series are stored with each significand in 3 digits or less.
    Generate a mapping from a 3-digit integer value to a value series.
    """

    # Value Series: "E6"
    E6_str = """
    10	15	22	33	47	68
    """

    # Value Series: "E12"...
    E12_str = """
    10	12	15	18	22	27
    33	39	47	56	68	82
    """

    E24_str = """
    10	11	12	13	15	16
    18	20	22	24	27	30
    33	36	39	43	47	51
    56	62	68	75	82	91
    """

    E48_str = """
    100	105	110	115	121	127
    133	140	147	154	162	169
    178	187	196	205	215	226
    237	249	261	274	287	301
    316	332	348	365	383	402
    422	442	464	487	511	536
    562	590	619	649	681	715
    750	787	825	866	909	953
    """

    E96_str = """
    100	102	105	107	110	113
    115	118	121	124	127	130
    133	137	140	143	147	150
    154	158	162	165	169	174
    178	182	187	191	196	200
    205	210	215	221	226	232
    237	243	249	255	261	267
    274	280	287	294	301	309
    316	324	332	340	348	357
    365	374	383	392	402	412
    422	432	442	453	464	475
    487	499	511	523	536	549
    562	576	590	604	619	634
    649	665	681	698	715	732
    750	768	787	806	825	845
    866	887	909	931	953	976
    """

    E192_str = """
    100	101	102	104	105	106	107	109	110	111	113	114
    115	117	118	120	121	123	124	126	127	129	130	132
    133	135	137	138	140	142	143	145	147	149	150	152
    154	156	158	160	162	164	165	167	169	172	174	176
    178	180	182	184	187	189	191	193	196	198	200	203
    205	208	210	213	215	218	221	223	226	229	232	234
    237	240	243	246	249	252	255	258	261	264	267	271
    274	277	280	284	287	291	294	298	301	305	309	312
    316	320	324	328	332	336	340	344	348	352	357	361
    365	370	374	379	383	388	392	397	402	407	412	417
    422	427	432	437	442	448	453	459	464	470	475	481
    487	493	499	505	511	517	523	530	536	542	549	556
    562	569	576	583	590	597	604	612	619	626	634	642
    649	657	665	673	681	690	698	706	715	723	732	741
    750	759	768	777	787	796	806	816	825	835	845	856
    866	876	887	898	909	920	931	942	953	965	976	988
    """

    

    def map_significand3_to_series(significand3_str, series, significand3_map):
        for significand3_str in significand3_str.split():
            significand3 = int(significand3_str.ljust(3, '0'))
            significand3_map[significand3] = series
        return significand3_map
    
    significand3_map = {}
    map_significand3_to_series(E192_str, 192, significand3_map)
    map_significand3_to_series(E96_str, 96, significand3_map)
    map_significand3_to_series(E48_str, 48, significand3_map)
    map_significand3_to_series(E24_str, 24, significand3_map)
    map_significand3_to_series(E12_str, 12, significand3_map)
    map_significand3_to_series(E6_str, 6, significand3_map)

    # Sort by key and return new dict
    return dict(sorted(significand3_map.items()))

significand3_map = gen_significand3_map()
print(significand3_map)


{100: 6, 101: 192, 102: 96, 104: 192, 105: 48, 106: 192, 107: 96, 109: 192, 110: 24, 111: 192, 113: 96, 114: 192, 115: 48, 117: 192, 118: 96, 120: 12, 121: 48, 123: 192, 124: 96, 126: 192, 127: 48, 129: 192, 130: 24, 132: 192, 133: 48, 135: 192, 137: 96, 138: 192, 140: 48, 142: 192, 143: 96, 145: 192, 147: 48, 149: 192, 150: 6, 152: 192, 154: 48, 156: 192, 158: 96, 160: 24, 162: 48, 164: 192, 165: 96, 167: 192, 169: 48, 172: 192, 174: 96, 176: 192, 178: 48, 180: 12, 182: 96, 184: 192, 187: 48, 189: 192, 191: 96, 193: 192, 196: 48, 198: 192, 200: 24, 203: 192, 205: 48, 208: 192, 210: 96, 213: 192, 215: 48, 218: 192, 220: 6, 221: 96, 223: 192, 226: 48, 229: 192, 232: 96, 234: 192, 237: 48, 240: 24, 243: 96, 246: 192, 249: 48, 252: 192, 255: 96, 258: 192, 261: 48, 264: 192, 267: 96, 270: 12, 271: 192, 274: 48, 277: 192, 280: 96, 284: 192, 287: 48, 291: 192, 294: 96, 298: 192, 300: 24, 301: 48, 305: 192, 309: 96, 312: 192, 316: 48, 320: 192, 324: 96, 328: 192, 330: 6, 332: 48, 336: 192, 34

In [123]:
def val_to_scinot(value):
    scinot_str = f"{value:e}"
    scinot_split_str = scinot_str.split('e')
    scinot_split_str[1] = scinot_split_str[1].replace('+','') # Replace plus sign added to positive exponents (if present)
    return (float(scinot_split_str[0]), int(scinot_split_str[1]))

class SeriesValue:
    """
    Generic class for storing values that belong to a component series.
    """
    def __init__(self, series, value):
        self.series = series
        self.value = value
        self.significand, self.exponent = val_to_scinot(value)

    def from_significand_exponent(series, significand, exponent):
        return SeriesValue(series=series, value=significand*pow(10, exponent))

    def __repr__(self):
        if isinstance(self.series, (int, float)):
            series = f'E{self.series}'
        else:
            series = self.series
        return f"series={series}: {self.value:.3}"


def gen_series_values_in_range(min_value, max_value, accepted_series_list, significand3_map):
    # Get scientific notation representation of min and max
    min_significand, min_expononent = val_to_scinot(min_value)
    max_significand, max_expononent = val_to_scinot(max_value)

    # Convert significand to 3 digit integer for easier comparison later (avoid comparing calculated floats for equality)
    min_significand3 = int(round(min_significand*100))
    max_significand3 = int(round(max_significand*100))

    # make a list of all SeriesValues between min and max
    series_value_list = []

    # For each exponent within min and max range
    for exponent in range(min_expononent, max_expononent+1):
        # Inspect each 3-digit significand of an acceptable component value series
        for significand3, series in significand3_map.items():
            if series in accepted_series_list:
                # If we are on the lowest or highest possible exponent, compare min and max significands too
                # Or if looking at an exponent between max and min exponent, then it automatically qualifies as a valid value
                if (exponent == min_expononent and significand3 >= min_significand3) or \
                (exponent == max_expononent and significand3 <= max_significand3) or \
                (exponent > min_expononent and exponent < max_expononent):
                    series_value_list.append(
                        SeriesValue.from_significand_exponent(
                            series=series,
                            significand=significand3/100.0,
                            exponent=exponent
                        )
                    )
    return series_value_list


# gen_series_values_in_range(1e-6, 20e-5, accepted_series_list=[6, 12], significand3_map=significand3_map)

# Generic Component Classes with Parameters to Match DBLib

In [125]:
# Generate base class for component in database to help enforce parameters needed
class ComponentBase:
    def __init__(
        self,
        description,
        comment,
        manufacturer,
        manufacturer_pn,
        symbol_path,
        symbol_ref,
        footprint_path,
        footprint_ref,
        *args,
        **kwargs):

        self.description = description
        self.comment = comment
        self.manufacturer = manufacturer
        self.manufacturer_pn = manufacturer_pn
        self.symbol_path = symbol_path
        self.symbol_ref = symbol_ref
        self.footprint_path = footprint_path
        self.footprint_ref = footprint_ref
        for arg in args:
            print(f"Warning: Unassigned argument: {arg}")
        for key, val in kwargs.items():
            print(f"Warning: Unassigned key-val pair: {key} : {val}")

    def to_dict(self):
        # https://www.altium.com/documentation/altium-designer/creating-defining-database-library
        d = {}
        d['Description'] = self.description
        d['Comment'] = self.comment
        d['Manufacturer'] = self.manufacturer
        d['Manufacturer Part Number'] = self.manufacturer_pn
        d['Library Path'] = self.symbol_path
        d['Library Ref'] = self.symbol_ref
        d['Footprint Path'] = self.footprint_path
        d['Footprint Ref'] = self.footprint_ref
        return d

class ValuedComponent(ComponentBase):
    def __init__(
        self,
        value,
        tolerance,
        *args,
        **kwargs):

        super().__init__(*args, **kwargs)
        self.value = value
        self.tolerance = tolerance
    
    def to_dict(self):
        d = super().to_dict()
        d['Value'] = self.value
        d['Tolerance'] = self.tolerance
        return d

class ChipComponent(ValuedComponent):
    def __init__(
        self,
        size,
        *args,
        **kwargs):

        super().__init__(*args, **kwargs)
        self.size = size
    
    def to_dict(self):
        d = super().to_dict()
        d['Package / Case'] = self.size
        return d

# R = ValuedComponent(
R = ChipComponent(
    value='5R',
    tolerance='1%',
    size='0804',
    description='dscrpt',
    comment='comment',
    manufacturer='mfg',
    manufacturer_pn='pn',
    symbol_path='sympath',
    symbol_ref='symref',
    footprint_path='footpath',
    footprint_ref='footref'
)
R.to_dict()

{'Description': 'dscrpt',
 'Comment': 'comment',
 'Manufacturer': 'mfg',
 'Manufacturer Part Number': 'pn',
 'Library Path': 'sympath',
 'Library Ref': 'symref',
 'Footprint Path': 'footpath',
 'Footprint Ref': 'footref',
 'Value': '5R',
 'Tolerance': '1%'}

# Yageo RC_L Series

In [126]:
class Yageo_RC_L:
    sizes = [
        '0075',
        '0100',
        '0201',
        '0402',
        '0603',
        '0805',
        '1206',
        '1210',
        '1218',
        '2010',
        '2512'
    ]
    tolerances = {
        'B' : '0.1%',
        'D' : '0.5%',
        'F' : '1.0%',
        'J' : '5.0%' # Also for jumpers
    }
    packagings = {
        'R' : 'Paper taping reel',
        'K' : 'Embossed taping reel',
        'S' : 'ESD safe reel (0075/0100 only)'
    }
    reel_powers = {
        '07' : '7 inch dia. Reel & Standard power',
        '10' : '10 inch dia. Reel',
        '13' : '13 inch dia. Reel',
        '7W' : '7 inch dia. Reel & 2 x standard power',
        '7N' : '7 inch dia. Reel, ESD safe reel (0075/0100 only)',
        '3W' : '13 inch dia. Reel & 2 x standard power'
    }

    def gen_str(size, tolerance, packaging, reel_power, resistance):
        return f'RC{size}{tolerance}{packaging}-{reel_power}{resistance}L'

    def gen_properties(size, tolerance, packaging, reel_power, resistance_str):
        d = {}
        d['Package / Case'] = size
        if resistance_str == '0R':
            d['Tolerance'] = 'Jumper'
        else:
            d['Tolerance'] = Yageo_RC_L.tolerances[tolerance]
        d['Packaging'] = Yageo_RC_L.packagings[packaging]

        power_factor = 1
        if 'W' in reel_power:
            power_factor = 2

        if size == '0075':
            d['Power Rating'] = '1/50W'
        elif size == '0100':
            d['Power Rating'] = '1/32W'
        elif size == '0201':
            d['Power Rating'] = '1/20W'
        elif size == '0402':
            if 'W' not in reel_power:
                d['Power Rating'] = '1/16W'
            else:
                d['Power Rating'] = '1/8W'
        elif size == '0603':
            if 'W' not in reel_power:
                d['Power Rating'] = '1/10W'
            else:
                d['Power Rating'] = '1/5W'
        elif size == '0805':
            if 'W' not in reel_power:
                d['Power Rating'] = '1/8W'
            else:
                d['Power Rating'] = '1/4W'
        elif size == '1206':
            if 'W' not in reel_power:
                d['Power Rating'] = '1/4W'
            else:
                d['Power Rating'] = '1/2W'
        elif size == '1210':
            d['Power Rating'] = '1/2W'
        elif size == '1218':
            d['Power Rating'] = '1W'
        elif size == '2010':
            d['Power Rating'] = '3/4W'
        elif size == '2512':
            if 'W' not in reel_power:
                d['Power Rating'] = '1W'
            else:
                d['Power Rating'] = '2W'
        else:
            d['Power Rating'] = ''
        if resistance_str == '0R':
            d['Power Rating'] = 'Jumper'

        d['Value'] = resistance_str
        return d

In [127]:
size = '0805'
tolerance = 'F'
packaging = 'R'
reel_power = '07'

# Generate list of resistance strings
resistances = [
    {''}
]
for exponent in range(0, 9):
    for significand_3 in all_resistances:    
        # Determine resistance string
        resistance_str = ''
        if exponent < 3:
            resistances.append(significand_3[:exponent+1] + 'R' + significand_3[exponent+1:].rstrip('0'))
        elif exponent < 6:
            resistances.append(significand_3[:exponent-2] + 'K' + significand_3[exponent-2:].rstrip('0'))
        elif exponent < 9:
            resistances.append(significand_3[:exponent-2] + 'M' + significand_3[exponent-2:].rstrip('0'))
# print(resistances)

for resistance in resistances:
    # Generate component propoerties
    d = Yageo_RC_L.gen_properties(size, tolerance, packaging, reel_power, resistance)

    # Determine base series and add as parameter
    significand_3 = resistance
    significand_3 = significand_3.replace('R','')
    significand_3 = significand_3.replace('K','')
    significand_3 = significand_3.replace('M','')
    significand_3 = significand_3.ljust(3, '0')
    if significand_3 == '0':
        value_series = 'Jumper'
    elif significand_3 in E6:
        value_series = 'E6'
    elif significand_3 in E12:
        value_series = 'E12'
    elif significand_3 in E24:
        value_series = 'E24'
    elif significand_3 in E48:
        value_series = 'E48'
    else:
        value_series = 'E96'   
    d['Value Series'] = value_series

    d['Library Path'] = 'Symbols.SchLib'
    d['Library Ref'] = 'Resistor'
    d['Footprint Path'] = 'Footprints.PcbLib'
    d['Footprint Ref'] = size
    
    print(d)

NameError: name 'all_resistances' is not defined

In [ ]:
'a'.ljust(3, '0')

'a00'